In [3]:
import pandas as pd

PATH = "../executions/MNQM2022/20220318-20220329.csv"


def get_executions_df(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    df_grouped = df.groupby("date").sum()[["realized_pnl", "px_side"]]

    df_grouped["pnl / side"] = df_grouped["realized_pnl"] / df_grouped["px_side"]
    df_grouped["profit"] = df[df["realized_pnl"] > 0].groupby("date").count()["realized_pnl"]
    df_grouped["loss"] = df[df["realized_pnl"] < 0].groupby("date").count()["realized_pnl"]
    df_grouped.loc["Total"] = df_grouped.sum(numeric_only=True)

    return df_grouped


get_executions_df(PATH)

,realized_pnl,px_side,pnl / side,profit,loss
date,,,,,
2022-03-18,2326.020001,320.211667,7.264008,14.0,5.0
2022-03-21,-1456.600001,-306.435834,4.753361,12.0,15.0
2022-03-22,943.200000,117.325000,8.039207,3.0,2.0
2022-03-23,-261.500000,-47.263333,5.532830,5.0,9.0
2022-03-24,717.280000,145.170000,4.940966,6.0,3.0
2022-03-25,-69.179999,-76.455000,0.904846,17.0,12.0
2022-03-28,649.840000,53.153333,12.225762,11.0,8.0
2022-03-29,419.240000,47.706667,8.787870,13.0,5.0
Total,3268.300001,253.412500,52.448850,81.0,59.0


In [4]:
def get_executions_ungroup(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    cols = ["quantity", "realized_pnl", "px_side"]

    return (
        df,
        df.groupby("date").sum()[cols],
        df.groupby("date").count()[["realized_pnl"]]
        # Only a single column is needed for showing the total realization count
    )


df_all, df_sum, df_count = get_executions_ungroup(PATH)

In [5]:
df_sum


,quantity,realized_pnl,px_side
date,,,
2022-03-18,57,2326.020001,320.211667
2022-03-21,90,-1456.600001,-306.435834
2022-03-22,20,943.200000,117.325000
2022-03-23,50,-261.500000,-47.263333
2022-03-24,23,717.280000,145.170000
2022-03-25,87,-69.179999,-76.455000
2022-03-28,69,649.840000,53.153333
2022-03-29,59,419.240000,47.706667


In [6]:
df_count


,realized_pnl
date,
2022-03-18,19
2022-03-21,27
2022-03-22,5
2022-03-23,14
2022-03-24,9
2022-03-25,29
2022-03-28,19
2022-03-29,18


In [7]:
def get_executions_ungroup(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    return (
        df[df["realized_pnl"] > 0].groupby("date").sum()[["realized_pnl"]],
        df[df["realized_pnl"] < 0].groupby("date").sum()[["realized_pnl"]]
    )


df_pos, df_neg = get_executions_ungroup(PATH)

In [8]:
df_pos


,realized_pnl
date,
2022-03-18,2943.506668
2022-03-21,3122.900000
2022-03-22,1809.460000
2022-03-23,1888.120000
2022-03-24,1050.760000
2022-03-25,2184.020001
2022-03-28,2311.200000
2022-03-29,1323.340000


In [9]:
df_neg


,realized_pnl
date,
2022-03-18,-617.486667
2022-03-21,-4579.500001
2022-03-22,-866.260000
2022-03-23,-2149.620000
2022-03-24,-333.480000
2022-03-25,-2253.200000
2022-03-28,-1661.360000
2022-03-29,-904.100000
